In [116]:
import pandas as pd
import warnings
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams, bigrams, trigrams
#using TextBlob
from textblob import TextBlob

In [117]:
df_diabetes=pd.read_csv('tweets_diabetes.csv')

In [118]:
df_cancer=pd.read_csv('tweets_cancer.csv')

In [119]:
df_migraine=pd.read_csv('tweets_migraine.csv')

In [120]:
df_thyroid=pd.read_csv('tweets_thyroid.csv')

In [121]:
df_cancer.head()

,id,Time,Tweet,Location
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!
1,842288144909234176,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England"
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe
3,1337135104200757248,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN


In [122]:
df_thyroid.head()

,id,Time,Tweet,Location
0,1921659636,2022-11-09 11:05:39+00:00,"b'This is How it starts. Yikes, partial functi...",NaN
1,167121831,2022-11-09 11:03:21+00:00,b'RT @EDCFree: A \xf0\x9f\x86\x95 study from o...,Scotland
2,955518357771554819,2022-11-09 11:02:44+00:00,b'RT @ptcbtestprep: \xf0\x9f\x92\x8aThyroid Me...,"Punjab, Pakistan"
3,958761967665274880,2022-11-09 11:02:20+00:00,b'RT @SummrWrites: These are all facts. Too ma...,"North Carolina, USA"
4,1247186408835432448,2022-11-09 11:00:26+00:00,b'Consume this juice to keep thyroid under con...,"Uttar Pradesh, India"


In [123]:
df_migraine.head()

,id,Time,Tweet,Location
0,3819963134,2022-11-09 11:11:19+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",NaN
1,1553022141867917312,2022-11-09 11:10:40+00:00,b'RT @MariaMari0nette: I did it \xf0\x9f\xa4\x...,我非常爱他，我想嫁给他👟☯️
2,990262668526157824,2022-11-09 11:10:34+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",NaN
3,207334486,2022-11-09 11:10:31+00:00,b'My migraine at the moment https://t.co/oGwvI...,"North Carolina, USA"
4,14114178,2022-11-09 11:10:25+00:00,b'Ok so the first dose is done. I really hope ...,NaN


In [124]:

df_cancer['Class']='cancer'
df_thyroid['Class']='thyroid'
df_diabetes['Class']='diabetes'
df_migraine['Class']='migraine'

In [125]:
df1=pd.concat([df_cancer,df_thyroid,df_diabetes,df_migraine])

In [126]:
df1['Class'].unique()

array(['cancer', 'thyroid', 'diabetes', 'migraine'], dtype=object)

In [127]:
df1.to_csv('data.csv')

PermissionError: [Errno 13] Permission denied: 'data.csv'

In [128]:
df2=pd.read_csv('data.csv')

In [129]:
df2.head()

,id,Time,Tweet,Location,Class
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer


In [130]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        334 non-null    object
 1   Time      334 non-null    object
 2   Tweet     334 non-null    object
 3   Location  194 non-null    object
 4   Class     334 non-null    object
dtypes: object(5)
memory usage: 13.2+ KB


In [131]:
stop_words_file = 'SmartStoplist.txt'

stop_words = []

with open(stop_words_file, "r") as f:
    for line in f:
        stop_words.extend(line.split()) 
        
stop_words = stop_words

In [132]:
def preprocess(raw_text):
    
    #regular expression keeping only letters 
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split into words -> convert string into list ( 'hello world' -> ['hello', 'world'])
    words = letters_only_text.lower().split()

    cleaned_words = []
    lemmatizer = PorterStemmer() #plug in here any other stemmer or lemmatiser you want to try out
    
    # remove stopwords
    for word in words:
        if word not in stop_words:
            cleaned_words.append(word)
    
    # stemm or lemmatise words
    stemmed_words = []
    for word in cleaned_words:
        word = lemmatizer.stem(word)   #dont forget to change stem to lemmatize if you are using a lemmatizer
        stemmed_words.append(word)
    
    # converting list back to string
    return " ".join(stemmed_words)

In [133]:
df2['prep'] = df2['Tweet'].apply(preprocess)  


In [134]:
df2.head()

,id,Time,Tweet,Location,Class,prep
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...


In [135]:
df2['split_prep'] = df2['prep'].str.split(' ')

In [136]:
df2.head()

,id,Time,Tweet,Location,Class,prep,split_prep
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"[rt, theironmaiden, xe, bc, xc, ncer, xe, fing..."
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"[rt, annett, hardi, juststop, oil, consult, su..."
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"[rt, mutt, muttm, leo, cancer, pisc, aquariu, ..."
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"[jou, kaiser, efecto, secundario, de, la, mal,..."
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"[nconcostrina, pablocasado, es, mejor, mandar,..."


In [ ]:
df2.head()

In [138]:
df2.head()

,id,Time,Tweet,Location,Class,prep,split_prep
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"[rt, theironmaiden, xe, bc, xc, ncer, xe, fing..."
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"[rt, annett, hardi, juststop, oil, consult, su..."
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"[rt, mutt, muttm, leo, cancer, pisc, aquariu, ..."
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"[jou, kaiser, efecto, secundario, de, la, mal,..."
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"[nconcostrina, pablocasado, es, mejor, mandar,..."


In [139]:
#textblob

In [ ]:
df2=pd.read_csv('data.csv')

In [145]:
def sentiment_analysis(df):
    
    def getSubjectivity(text):
        
        return TextBlob(text).sentiment.subjectivity
    def getPolarity(text):
        
        return TextBlob(text).sentiment.polarity
    df['TextBlob_Subjectivity']=df['prep'].apply(getSubjectivity)
    df['TextBlob_Polarity']=df['prep'].apply(getPolarity)
    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'
        df['TextBlob_Analysis'] = df['TextBlob_Polarity'].apply(getAnalysis )
    return df

In [146]:
sentiment_analysis(df2)

,id,Time,Tweet,Location,Class,prep,split_prep,TextBlob_Subjectivity,TextBlob_Polarity
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"[rt, theironmaiden, xe, bc, xc, ncer, xe, fing...",0.0,0.0
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"[rt, annett, hardi, juststop, oil, consult, su...",0.0,0.0
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"[rt, mutt, muttm, leo, cancer, pisc, aquariu, ...",0.0,0.0
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"[jou, kaiser, efecto, secundario, de, la, mal,...",0.0,0.0
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"[nconcostrina, pablocasado, es, mejor, mandar,...",0.0,0.0
...,...,...,...,...,...,...,...,...,...
329,1.50654E+18,2022-11-09 10:40:35+00:00,b'RT @QueenBelindaMP: Migraine...ughhhhh \xf0\...,NaN,migraine,rt queenbelindamp migrain ughhhhh xf xa xf xad...,"[rt, queenbelindamp, migrain, ughhhhh, xf, xa,...",0.0,0.0
330,1.39444E+18,2022-11-09 10:40:10+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",NaN,migraine,rt wipor mont worri migrain http hcrzp kk,"[rt, wipor, mont, worri, migrain, http, hcrzp,...",0.0,0.0
331,1.26399E+18,2022-11-09 10:39:45+00:00,b'finished the movie and now i have a migraine...,20 ! they/them,migraine,finish movi migrain unrel,"[finish, movi, migrain, unrel]",0.0,0.0
332,1.34071E+18,2022-11-09 10:37:38+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",Istanbul,migraine,rt wipor mont worri migrain http hcrzp kk,"[rt, wipor, mont, worri, migrain, http, hcrzp,...",0.0,0.0


In [151]:
df2.drop(['TextBlob_Subjectivity'],axis=1,inplace=True)

In [152]:
df2.drop(['TextBlob_Polarity'],axis=1,inplace=True)

In [153]:
df2.head()

,id,Time,Tweet,Location,Class,prep,split_prep
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"[rt, theironmaiden, xe, bc, xc, ncer, xe, fing..."
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"[rt, annett, hardi, juststop, oil, consult, su..."
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"[rt, mutt, muttm, leo, cancer, pisc, aquariu, ..."
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"[jou, kaiser, efecto, secundario, de, la, mal,..."
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"[nconcostrina, pablocasado, es, mejor, mandar,..."


In [156]:
df2.to_csv('data11.csv')

In [157]:
df3=pd.read_csv('data11.csv')

In [158]:
df3.head()

,Unnamed: 0,id,Time,Tweet,Location,Class,prep,split_prep
0,0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"['rt', 'theironmaiden', 'xe', 'bc', 'xc', 'nce..."
1,1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"['rt', 'annett', 'hardi', 'juststop', 'oil', '..."
2,2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"['rt', 'mutt', 'muttm', 'leo', 'cancer', 'pisc..."
3,3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"['jou', 'kaiser', 'efecto', 'secundario', 'de'..."
4,4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"['nconcostrina', 'pablocasado', 'es', 'mejor',..."


In [160]:
df3.drop(['Unnamed: 0'],axis=1,inplace=True)

In [161]:
df=pd.read_csv('data11.csv')

In [162]:
df.head()

,Unnamed: 0,id,Time,Tweet,Location,Class,prep,split_prep
0,0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"['rt', 'theironmaiden', 'xe', 'bc', 'xc', 'nce..."
1,1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"['rt', 'annett', 'hardi', 'juststop', 'oil', '..."
2,2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"['rt', 'mutt', 'muttm', 'leo', 'cancer', 'pisc..."
3,3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"['jou', 'kaiser', 'efecto', 'secundario', 'de'..."
4,4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"['nconcostrina', 'pablocasado', 'es', 'mejor',..."


In [163]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [164]:
df.head()

,id,Time,Tweet,Location,Class,prep,split_prep
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"['rt', 'theironmaiden', 'xe', 'bc', 'xc', 'nce..."
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"['rt', 'annett', 'hardi', 'juststop', 'oil', '..."
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"['rt', 'mutt', 'muttm', 'leo', 'cancer', 'pisc..."
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"['jou', 'kaiser', 'efecto', 'secundario', 'de'..."
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"['nconcostrina', 'pablocasado', 'es', 'mejor',..."


In [165]:
def sentiment_analysis(df):
    
    def getSubjectivity(text):
        
        return TextBlob(text).sentiment.subjectivity
    def getPolarity(text):
        
        return TextBlob(text).sentiment.polarity
    df['TextBlob_Subjectivity']=df['prep'].apply(getSubjectivity)
    df['TextBlob_Polarity']=df['prep'].apply(getPolarity)
    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'
        df['TextBlob_Analysis'] = df['TextBlob_Polarity'].apply(getAnalysis )
    return df

In [166]:
sentiment_analysis(df)

,id,Time,Tweet,Location,Class,prep,split_prep,TextBlob_Subjectivity,TextBlob_Polarity
0,1242402950,2022-11-09 11:09:56+00:00,b'RT @_TheIronMaiden_: \xe2\x99\x8bC\xc3\x81NC...,Ande andaré?!,cancer,rt theironmaiden xe bc xc ncer xe fingolfin xe...,"['rt', 'theironmaiden', 'xe', 'bc', 'xc', 'nce...",0.0,0.0
1,8.42288E+17,2022-11-09 11:09:55+00:00,"b'RT @Annette1Hardy: Well done, @JustStop_Oil....","South East, England",cancer,rt annett hardi juststop oil consult surgeon f...,"['rt', 'annett', 'hardi', 'juststop', 'oil', '...",0.0,0.0
2,4086112344,2022-11-09 11:09:55+00:00,"b'RT @Mutt_muttm: Leo, cancer, pisces , aquari...",universe,cancer,rt mutt muttm leo cancer pisc aquariu gemini c...,"['rt', 'mutt', 'muttm', 'leo', 'cancer', 'pisc...",0.0,0.0
3,1.33714E+18,2022-11-09 11:09:55+00:00,b'@Jou_Kaiser EFECTOS SECUNDARIOS DE LAS MAL L...,NaN,cancer,jou kaiser efecto secundario de la mal llamada...,"['jou', 'kaiser', 'efecto', 'secundario', 'de'...",0.0,0.0
4,386097138,2022-11-09 11:09:55+00:00,b'@NConcostrina @pablocasado_ Y no es mejor ma...,NaN,cancer,nconcostrina pablocasado es mejor mandar esto ...,"['nconcostrina', 'pablocasado', 'es', 'mejor',...",0.0,0.0
...,...,...,...,...,...,...,...,...,...
329,1.50654E+18,2022-11-09 10:40:35+00:00,b'RT @QueenBelindaMP: Migraine...ughhhhh \xf0\...,NaN,migraine,rt queenbelindamp migrain ughhhhh xf xa xf xad...,"['rt', 'queenbelindamp', 'migrain', 'ughhhhh',...",0.0,0.0
330,1.39444E+18,2022-11-09 10:40:10+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",NaN,migraine,rt wipor mont worri migrain http hcrzp kk,"['rt', 'wipor', 'mont', 'worri', 'migrain', 'h...",0.0,0.0
331,1.26399E+18,2022-11-09 10:39:45+00:00,b'finished the movie and now i have a migraine...,20 ! they/them,migraine,finish movi migrain unrel,"['finish', 'movi', 'migrain', 'unrel']",0.0,0.0
332,1.34071E+18,2022-11-09 10:37:38+00:00,"b""RT @Wipor_mont: Don't worry, it is just a mi...",Istanbul,migraine,rt wipor mont worri migrain http hcrzp kk,"['rt', 'wipor', 'mont', 'worri', 'migrain', 'h...",0.0,0.0
